In [ ]:
import os
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/figures/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

dn = smooth.analysis.get_display_names()

In [ ]:
LOG_DIR = "/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/"

ms  = pd.read_feather(os.path.join(LOG_DIR, "0326_mnist_binary/measures.feather"))
ms2 = pd.read_feather(os.path.join(LOG_DIR, "0329_mnist_binary_gradient_norm/measures.feather"))
ms3 = pd.read_feather(os.path.join(LOG_DIR, "0331_mnist_binary_weights_product/measures.feather"))

ms = pd.concat([ms, ms2, ms3], sort=False)
ms = ms.reset_index(drop=True)

print("Removing {} entries".format(sum(ms["gradient_norm_test"].isna())))
ms = ms[~ms["gradient_norm_test"].isna()]
ms["model.weights_product_reg_coef"] = ms["model.weights_product_reg_coef"].fillna(value=0)

smooth.analysis.remove_constant_columns(ms, verbose=True)

ms["kind"] = "normal"
for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.weights_product_reg_coef"] == val, "kind"] = "weights_product_{:.0e}".format(val)
# ms.loc[ms["model.weights_product_reg_coef"] > 0, "kind"] = "weights_product"

for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.gradient_norm_reg_coef"] == val, "kind"] = "gradient_norm_{:.0e}".format(val)

# Add normalized gradient norm
ms1 = ms[ms["kind"] == "normal"].copy()
ms1 = ms1.rename(columns={"gradient_norm_test": "gradient_norm_test_unreg"})
ms = pd.merge(
    ms,
    ms1[["dataset.name", "model.hidden_size", "gradient_norm_test_unreg"]],
    on=["dataset.name", "model.hidden_size"],
)

ms["gradient_norm_test_normalized"] = ms["gradient_norm_test"] / ms["gradient_norm_test_unreg"]
del ms["gradient_norm_test_unreg"]


In [ ]:
def should_plot_as_log(measure_name):
    patterns = [
        "loss",
#         "weights_product",
    ]
    
    for p in patterns:
        if p in measure_name:
            return True
    
    return False

def plot_measure(ms1, measure_name, groupby):
    ms1 = ms1.rename(columns=dn)
    
    grid = sns.lineplot(
        data=ms1,
        x=dn[groupby],
        y=dn[measure_name],
        hue=dn[measure_name],
        palette=smooth.analysis.make_palette(ms1[dn[measure_name]]),
#         col="kind",
#         hue_order=sorted(ms1["kind"].unique()),
#         kind="line",
    )
    ax = grid
#     ax = grid.axes[0][0]
    if "loss" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim((3, 1200))
    ax.set_xticks(sorted(ms[groupby].unique()))
#     ax.get_xaxis().get_major_formatter().set_scientific(False)
    ax.xaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())

    for t in ax.get_legend().texts[1:]:
#         print(t.get_text())
        t.set_text("${}$".format(smooth.analysis.to_scientific_tex(float(t.get_text()))))
    plt.show()
    

def plot_measure(ms, measure_name, groupby):
    ms = ms.copy()
    ms = ms.rename(columns=dn)
    
    if False and should_plot_as_log(measure_name):
        log_measure_name = "log10_{}".format(measure_name)
        ms[log_measure_name] = np.log10(ms[measure_name])
        measure_name = log_measure_name

    sns.boxplot(data=ms, x=dn[measure_name], y=dn[groupby], orient="h")
    swarm_plot = sns.swarmplot(data=ms, x=dn[measure_name], y=dn[groupby],
                      size=2, color=".3", linewidth=0, orient="h",
                 )
    
#     for t in swarm_plot.get_yticklabels():
#         print(float(t.get_text()))
#         t.set_text("${}$".format(smooth.analysis.to_scientific_tex(float(t.get_text()))))
#         t.set_text("foo")
    
    swarm_plot.set_yticklabels([
        "${}$".format(smooth.analysis.to_scientific_tex(float(t.get_text())))
        for t in swarm_plot.get_yticklabels()
    ])
    
#     if xlabel:
#         swarm_plot.set(xlabel=xlabel)
    plt.show()
    fig = swarm_plot.get_figure()
    
    return fig

def describe(ms, reg_col, reg_coef, measure_col):
    return ms.loc[ms[reg_col] == reg_coef, measure_col].describe()

def describe_all(ms, reg_col, reg_coef, measure_col, normalized_measure_col=None):
    if normalized_measure_col is None:
        normalized_measure_col = measure_col + "_normalized"

    unreg = describe(ms, reg_col, 0, measure_col)
    reg = describe(ms, reg_col, reg_coef, measure_col)
    reg_normalized = describe(ms, reg_col, reg_coef, normalized_measure_col)
    
    print("Unregularized mean:    {:.2f}".format(unreg["mean"]))
    print("Regularized mean:      {:.2f}".format(reg["mean"]))
    print("Normalized mean ± std: {:.2f} ± {:.2f}".format(reg_normalized["mean"], reg_normalized["std"]))
    print("Models dropped:        {}".format(int(unreg["count"] - reg["count"])))

## Reruns 

In [ ]:
# Updating rcParams before plotting anything breaks matplotlib for some reason
plt.plot([1], [2])

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

### Gradient norm

In [ ]:
ms = smooth.analysis.load_measures(
    "../logs/0410_gradient_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.gradient_norm_reg_coef", "gn"),
    ],
)

# plot_all_measures(ms)

In [ ]:
ms1 = ms[ms["model.learning_rate"] == 0.01].copy()
ms1["Regularization coef"] = ms1["model.gradient_norm_reg_coef"]

ms1 = smooth.analysis.get_ratios(ms1, ms1["model.gradient_norm_reg_coef"] == 0, "gradient_norm_test")

fig1 = plot_measure(ms1, "gradient_norm_test", groupby="model.gradient_norm_reg_coef")
fig1.show()
# fig1.savefig("../figures/gradient_norm_reg_gradient_norm_test.pdf", bbox_inches="tight")
# plot_all_measures(ms1, "Regularization coef")

In [ ]:
describe_all(ms1, "model.gradient_norm_reg_coef", 1e-3, "gradient_norm_test")

In [ ]:
fig2 = plot_measure(ms1, "gradient_norm_test_normalized",
                    groupby="model.gradient_norm_reg_coef")
fig2.show()
# fig2.savefig("../figures/gradient_norm_reg_gradient_norm_test_normalized.pdf", bbox_inches="tight")

### Weights product

In [ ]:
ms = smooth.analysis.load_measures(
    "../logs/0412_weights_product_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
)
ms = ms[ms["model.weights_product_reg_coef"] < 1e-4]

ms1 = smooth.analysis.load_measures(
    "../logs/0413_weights_product_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
)
ms = pd.concat([ms, ms1], sort=False)
ms = ms.sort_values(["model.learning_rate", "model.weights_product_reg_coef"])

# plot_all_measures(ms)

In [ ]:
ms1 = ms[ms["model.learning_rate"] == 0.01].copy()

ms1 = ms1[ms1["dataset.name"].isin(
    set(ms1.loc[ms1["model.weights_product_reg_coef"] == 3e-5, "dataset.name"])
)]

ms1 = smooth.analysis.get_ratios(ms1, ms1["model.weights_product_reg_coef"] == 0, "weights_product")

# ms1["Regularization coef"] = ms1["model.weights_product_reg_coef"].map(str)

fig = plot_measure(
    ms1,
    "weights_product",
    groupby="model.weights_product_reg_coef",
#     xlabel="Weights product",
)
fig.show()
# ax.set(xlabel='common xlabel', ylabel='common ylabel')
# fig.savefig("../figures/weights_product_reg_weights_product.pdf", bbox_inches="tight")
# plot_all_measures(ms1, "Regularization coef")

In [ ]:
describe_all(ms1, "model.weights_product_reg_coef", 3e-5, "weights_product")

In [ ]:
fig = plot_measure(
    ms1,
    "weights_product_normalized",
    groupby="model.weights_product_reg_coef",
#     xlabel="Weights product - normed",
)
fig.show()
fig.savefig("../figures/weights_product_reg_weights_product_normalized.pdf", bbox_inches="tight")
# fig.savefig("../figures/teatalk_weights_product_normalized.png", bbox_inches="tight")

### Function path length

In [ ]:
ms = smooth.analysis.load_measures(
    "../logs/0508_path_length_f_reg/measures.feather",
    kind_cols=[
        ("model.path_length_f_reg_coef", "coef"),
    ],
)

ms0 = smooth.analysis.load_measures(
    "../logs/0422_path_length_f_reg/measures.feather",
#     "./0421_path_length_f_reg/measures.feather",
    kind_cols=[
        ("model.path_length_f_reg_coef", "coef"),
    ],
)

ms0 = ms0[ms0["model.path_length_f_reg_coef"] == 0]
ms = pd.concat([ms0, ms], sort=False)

converged_datasets = ms[(ms["model.path_length_f_reg_coef"] == 0.0001)]
ms = ms[ms["dataset.name"].isin(converged_datasets["dataset.name"])]
ms["path_length_f_test_baselined"] = ms["path_length_f_test"] - 1

ms = smooth.analysis.get_ratios(ms, ms["model.path_length_f_reg_coef"] == 0, "path_length_f_test_baselined")

In [ ]:
describe_all(ms, "model.path_length_f_reg_coef", 1e-4, "path_length_f_test",
             normalized_measure_col="path_length_f_test_baselined_normalized")

In [ ]:
fig = plot_measure(
    ms,
    "path_length_f_test",
    groupby="model.path_length_f_reg_coef",
#     xlabel="Weights product",
)
fig.show()
fig.savefig("../figures/path_length_f_reg.pdf", bbox_inches="tight")

In [ ]:
ms = smooth.analysis.get_ratios(ms, ms["model.path_length_f_reg_coef"] == 0, "path_length_f_test")

fig = plot_measure(
    ms,
    "path_length_f_test_normalized",
    groupby="model.path_length_f_reg_coef",
#     xlabel="Weights product",
)
fig.show()

In [ ]:
fig = plot_measure(
    ms,
    "path_length_f_test_baselined_normalized",
    groupby="model.path_length_f_reg_coef",
#     xlabel="Weights product",
)
fig.show()
fig.savefig("../figures/path_length_f_reg_baselined_normalized.pdf", bbox_inches="tight")

### Gradient path length

In [ ]:
ms0 = smooth.analysis.load_measures(
    "../logs/0423_path_length_d_reg/measures.feather",
    kind_cols=[
        ("model.path_length_d_reg_coef", "coef"),
    ],
)

ms = smooth.analysis.load_measures(
    "../logs/0508_path_length_d_reg/measures.feather",
    kind_cols=[
        ("model.path_length_d_reg_coef", "coef"),
    ],
)

ms0 = ms0[ms0["dataset.name"].isin(set(ms["dataset.name"]))]

ms = pd.concat([ms0, ms], sort=False)
ms = smooth.analysis.get_ratios(ms, ms["model.path_length_d_reg_coef"] == 0, "path_length_d_test")

fig1 = plot_measure(
    ms,
    "path_length_d_test",
    groupby="model.path_length_d_reg_coef"
)
fig1.show()
# fig1.savefig("../figures/path_length_d_reg.pdf", bbox_inches="tight")

In [ ]:
describe_all(ms, "model.path_length_d_reg_coef", 1e-5, "path_length_d_test",
             normalized_measure_col="path_length_d_test_normalized")

In [ ]:
fig1 = plot_measure(
    ms,
    "path_length_d_test_normalized",
    groupby="model.path_length_d_reg_coef"
)
fig1.show()
# fig1.savefig("../figures/path_length_d_reg_normalized.pdf", bbox_inches="tight")

## Old

In [ ]:
ms1 = ms[ms["model.weights_product_reg_coef"] == 0]

for measure in [
            "loss_train",
            "loss_test",
            "gradient_norm_train",
            "gradient_norm_test",
            "gradient_norm_test_normalized",
#             "weights_product",
        ]:
    plt.figure()
    plot_measure(ms1[:10], measure, "model.gradient_norm_reg_coef")
#     plt.savefig("gradient_norm_reg_{}.pdf".format(measure), bbox_inches="tight")

In [ ]:
ms1 = ms[ms["model.gradient_norm_reg_coef"] == 0]

for measure in [
            "loss_train",
            "loss_test",
            "weights_product",
#             "gradient_norm_test_normalized",
#             "weights_product",
        ]:
    plt.figure()
    plot_measure(ms1[:10], measure, "model.weights_product_reg_coef")
#     plt.savefig("gradient_norm_reg_{}.pdf".format(measure), bbox_inches="tight")